In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Предисловие

Для парсинга был выбран сайт Drom.ru, а именно его владивостокский субдомен

С сайтом очень повезло - не потребовался даже фэйковый юзерагент, сервер покорно отдаёт страницы питону

Вытягивать будем информацию из объявлений о продаже автомобилей, но не всю, а только ключевые атрибуты, которые содержатся прямо в списке объявлений (не очень хочется нагружать сайт открытием каждого объявления, да и для учебного задания имеющейся информации вполне достаточно)

# Получение списка объявлений

Сперва обернём в функцию логику получения списка объявлений на странице

In [2]:
def get_ads_list(page_number):
    # задаём ссылку на страницу
    page_link = f'https://vladivostok.drom.ru/auto/all/page{page_number}'
    
    # пытаемся получить ответ сервера по ссылке. В случсе неудачи - возвращаем пустой список объявлений
    response = requests.get(page_link)
    if not response.ok:
        return [] 
    
    # в случае успешного получения ответа - берём код полченной страницы и преобразуем его в bs4
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # получаем из струткуры страницы список объявлений (20 шт)
    ads = soup.findAll('div', class_='b-advItem__inner')
    
    return ads

# Получение атрибутов объявления

Далее набросаем маленькие функции для получения отдельных атрибутов объявления, чтобы потом их использовать в цикле по списку объявлений

In [3]:
def get_model(ad):
    # разбираем заголовок объявления, получем модель
    title = ad.find('div', class_='b-advItem__title')
    model = title.text[:-6]
    return model

def get_year(ad):
    # разбираем заголовок объявления, получем год выпуска
    title = ad.find('div', class_='b-advItem__title')
    year = title.text[-4:]
    return year

def get_volume(ad):
    # разбираем первую строку из блока характеристик, получем объём двигателя
    volume_power = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_volume-power'})
    if volume_power:
        volume = ''.join([n for n in volume_power.contents[0] if n.isdigit()])
        volume = float(volume[:-1]+'.'+volume[-1:])
    else:
        volume = ''
    return volume

def get_power(ad):
    # разбираем первую строку из блока характеристик, получем мощность двигателя
    volume_power = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_volume-power'})
    power = int(''.join([n for n in volume_power.span.text if n.isdigit()])) if volume_power else ''
    return power

def get_fuel(ad):
    # разбираем вторую строку из блока характеристик, получем топливо
    fuel = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_fueltype'})
    fuel = fuel.text if fuel else 'гибрид'
    return fuel

def get_transmission(ad):
    # разбираем третью строку из блока характеристик, получаем трансмиссию
    transm = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_transmission'})
    transm = transm.text if transm else ''
    return transm

def get_weeldrive(ad):
    # разбираем четвёртую строку из блока характеристик, получаем привод
    wd = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_privod'})
    wd = wd.text if wd else ''
    return wd

def get_mileage(ad):
    # разбираем пятую строку из блока характеристик, получаем пробег
    miles = ad.find('div', attrs = {'data-ftid':'sales__bulls-item_mileage'})
    miles = int(''.join([n for n in miles.text if n.isdigit()])) if miles else ''
    return miles

def get_price(ad):
    # разбираем правый блок, получаем цену
    price = ad.find(lambda tag: tag.name == 'div' and tag.get('class') == ['b-advItem__price'])
    price = int(price.text.strip().replace('\xa0','')[:-1])
    return price

# Получение содержимого по списку объявлений

Здесь у нас функция, которая возвращает промежуточный датафрейм с информацией из объявлений на текущей странице

In [4]:
def get_ads_content(ads_list):
    ads_df = pd.DataFrame()
    for ad in ads_list:
        ad_row = {'model_name':get_model(ad),
                  'year_prduction':get_year(ad),
                  'engine_volume':get_volume(ad),
                  'engine_power':get_power(ad),
                  'fuel_type':get_fuel(ad),
                  'transmission':get_transmission(ad),
                  'driving_wheels':get_weeldrive(ad),
                  'mileage': get_mileage(ad),
                  'price':get_price(ad)}
        ads_df = ads_df.append(ad_row, ignore_index=True)
    return ads_df

# Собственно парсинг

In [5]:
pages = 100
result_df = pd.DataFrame()

for page in range(pages):
    result_df = result_df.append(get_ads_content(get_ads_list(page)))
result_df

,driving_wheels,engine_power,engine_volume,fuel_type,mileage,model_name,price,transmission,year_prduction
0,передний,98,1.2,бензин,,Nissan Note,390000.0,автомат,2012
1,передний,150,1.5,бензин,,Honda Stepwgn,1159000.0,автомат,2015
2,4WD,109,1.5,бензин,120,Nissan Wingroad,419000.0,автомат,2011
3,4WD,235,4.7,бензин,,Toyota Land Cruiser,950000.0,автомат,1999
4,4WD,270,4.7,бензин,180,Lexus GX470,1250000.0,автомат,2005
...,...,...,...,...,...,...,...,...,...
15,4WD,122,2.4,дизель,128,Land Rover Defender,900000.0,механика,2008
16,4WD,263,3.5,бензин,,Acura MDX,300000.0,автомат,2001
17,4WD,109,1.5,бензин,99,Nissan Wingroad,555000.0,автомат,2012
18,4WD,147,2,гибрид,80,Nissan X-Trail,1270000.0,автомат,2015


Получили вполне себе юзабельный датасет